In [ ]:
import numpy as np
import pandas as pd
import os, glob
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm, metrics
#from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import math
import gzip
import csv
import matplotlib.pyplot as plt
import psycopg2

In [ ]:
# path with all files
path = "C:/Users/mnigc/Documents/PCM/"
patient_ids=pd.read_csv('Matchedsubjectidnumerics.csv')
expired=patient_ids['Subject_id_dead'].dropna()
alive=patient_ids['Subject_id_alive'].dropna()

# get unique patient ids
unique_ids=np.unique(patient_ids)

# clear NaN
unique_ids=unique_ids[~np.isnan(unique_ids)]

len(unique_ids)

2597

# Unzipping mimic files

In [ ]:
# unzipping .gz version of admissions
admissions_unzip = gzip.open('ADMISSIONS.csv.gz', 'r')

# admissions file
admissions = pd.read_csv(admissions_unzip)

admissions=admissions[admissions['SUBJECT_ID'].isin(unique_ids)]

#print(admissions)

In [ ]:
# unzipping .gz version of patients
patients_unzip = gzip.open('PATIENTS.csv.gz','r')

# patients file
patients = pd.read_csv(patients_unzip)

patients=patients[patients['SUBJECT_ID'].isin(unique_ids)]

# Demographics/static-only feature extraction

In [ ]:
# Get ethnicities
ethnicities = admissions[['SUBJECT_ID','ETHNICITY']]
ethnicities=ethnicities.drop_duplicates(subset=['SUBJECT_ID'])
#ethnicities['ETHNICITY'].value_counts()

# one hot encoding
ethnicities = pd.get_dummies(ethnicities, columns=['ETHNICITY']) 

# ethnicities for patient subset
ethnicities=ethnicities[ethnicities['SUBJECT_ID'].isin(unique_ids)]





,SUBJECT_ID,ETHNICITY_ASIAN,ETHNICITY_ASIAN - ASIAN INDIAN,ETHNICITY_ASIAN - CHINESE,ETHNICITY_ASIAN - VIETNAMESE,ETHNICITY_BLACK/AFRICAN AMERICAN,ETHNICITY_BLACK/CAPE VERDEAN,ETHNICITY_HISPANIC OR LATINO,ETHNICITY_HISPANIC/LATINO - DOMINICAN,ETHNICITY_HISPANIC/LATINO - PUERTO RICAN,ETHNICITY_HISPANIC/LATINO - SALVADORAN,ETHNICITY_MULTI RACE ETHNICITY,ETHNICITY_OTHER,ETHNICITY_PATIENT DECLINED TO ANSWER,ETHNICITY_UNABLE TO OBTAIN,ETHNICITY_UNKNOWN/NOT SPECIFIED,ETHNICITY_WHITE,ETHNICITY_WHITE - BRAZILIAN,ETHNICITY_WHITE - EASTERN EUROPEAN,ETHNICITY_WHITE - RUSSIAN
5,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
10,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
15,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
19,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
21,41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15257,9372,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
15259,9374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
15260,9375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
15262,9377,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
ethnic_list = ["ASIAN", "BLACK", "HISPANIC", "WHITE"]
"""
df_test = ethnicities.filter(regex='ASIAN')
ty = df_test.sum(axis=1)
ethnicities["Asian"] = ty
ethnicities[ethnicities["Asian"] == 1]
"""
for ethnic in ethnic_list:
    hi = ethnicities.filter(regex=ethnic)
    ty = hi.sum(axis=1)
    ethnicities[ethnic] = ty
    cols_to_drop = list(hi.columns)
    ethnicities = ethnicities.drop(cols_to_drop, axis = 1)

ethnicities


,SUBJECT_ID,ETHNICITY_MULTI RACE ETHNICITY,ETHNICITY_OTHER,ETHNICITY_PATIENT DECLINED TO ANSWER,ETHNICITY_UNABLE TO OBTAIN,ETHNICITY_UNKNOWN/NOT SPECIFIED,Asian,ASIAN,BLACK,HISPANIC,WHITE
5,26,0,0,0,0,1,0,0,0,0,0
10,32,0,0,0,0,0,0,0,0,0,1
15,36,0,0,0,0,0,0,0,0,0,1
19,38,0,0,0,0,0,0,0,0,0,1
21,41,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
15257,9372,0,0,0,0,0,0,0,0,0,1
15259,9374,0,0,0,0,0,0,0,0,0,1
15260,9375,0,0,0,0,0,0,0,0,0,1
15262,9377,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Get marital status
marital_status=admissions[['SUBJECT_ID','MARITAL_STATUS']]
# One hot encoding
marital_status = pd.get_dummies(marital_status, columns=['MARITAL_STATUS'])
marital_status=marital_status.drop_duplicates(subset=['SUBJECT_ID'])
# Marital status for patient subset
marital_status=marital_status[marital_status['SUBJECT_ID'].isin(unique_ids)]


,SUBJECT_ID,MARITAL_STATUS_DIVORCED,MARITAL_STATUS_MARRIED,MARITAL_STATUS_SEPARATED,MARITAL_STATUS_SINGLE,MARITAL_STATUS_UNKNOWN (DEFAULT),MARITAL_STATUS_WIDOWED
5,26,0,0,0,1,0,0
10,32,0,0,0,0,0,0
15,36,0,1,0,0,0,0
19,38,0,0,0,0,0,1
21,41,0,0,0,1,0,0
...,...,...,...,...,...,...,...
15257,9372,0,1,0,0,0,0
15259,9374,0,1,0,0,0,0
15260,9375,0,1,0,0,0,0
15262,9377,0,1,0,0,0,0


In [ ]:
# Get admission type
admit_type=admissions[['SUBJECT_ID','ADMISSION_TYPE']]
# One hot encoding
admit_type = pd.get_dummies(admit_type, columns=['ADMISSION_TYPE']) 
admit_type=admit_type.drop_duplicates(subset=['SUBJECT_ID'])
# Admit type for patient subset
admit_type=admit_type[admit_type['SUBJECT_ID'].isin(unique_ids)]

,SUBJECT_ID,ADMISSION_TYPE_ELECTIVE,ADMISSION_TYPE_EMERGENCY,ADMISSION_TYPE_URGENT
5,26,0,1,0
10,32,1,0,0
15,36,0,1,0
19,38,0,1,0
21,41,1,0,0
...,...,...,...,...
15257,9372,0,1,0
15259,9374,0,1,0
15260,9375,0,1,0
15262,9377,0,1,0


In [ ]:
# Get admission location
admit_loc=admissions[['SUBJECT_ID','ADMISSION_LOCATION']]
# One hot encoding
admit_loc = pd.get_dummies(admit_loc, columns=['ADMISSION_LOCATION']) 
admit_loc=admit_loc.drop_duplicates(subset=['SUBJECT_ID'])
# Admission location for patient subset
admit_loc[admit_loc['SUBJECT_ID'].isin(unique_ids)]

,SUBJECT_ID,ADMISSION_LOCATION_CLINIC REFERRAL/PREMATURE,ADMISSION_LOCATION_EMERGENCY ROOM ADMIT,ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI,ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM,ADMISSION_LOCATION_TRANSFER FROM OTHER HEALT,ADMISSION_LOCATION_TRANSFER FROM SKILLED NUR
5,26,0,0,0,1,0,0
10,32,0,0,1,0,0,0
15,36,1,0,0,0,0,0
19,38,0,0,0,1,0,0
21,41,0,0,1,0,0,0
...,...,...,...,...,...,...,...
15257,9372,0,1,0,0,0,0
15259,9374,0,1,0,0,0,0
15260,9375,0,1,0,0,0,0
15262,9377,0,1,0,0,0,0


In [ ]:
# Get gender
gender=patients[['SUBJECT_ID','GENDER']]
# One hot encoding
gender = pd.get_dummies(gender, columns=['GENDER']) 
gender=gender.drop_duplicates(subset=['SUBJECT_ID'])
# Gender for patient subset
gender=gender[gender['SUBJECT_ID'].isin(unique_ids)]

,SUBJECT_ID,GENDER_F,GENDER_M
3,252,0,1
10,261,0,1
14,265,0,1
16,267,1,0
17,268,1,0
...,...,...,...
9842,9984,1,0
9844,9987,0,1
9848,9991,1,0
9850,9993,0,1


In [ ]:
# Get gender
insurance=admissions[['SUBJECT_ID','INSURANCE']]
# One hot encoding
insurance = pd.get_dummies(gender, columns=['INSURANCE']) 
insurance=insurance.drop_duplicates(subset=['SUBJECT_ID'])
# insurance for patient subset
insurance=insurance[insurance['SUBJECT_ID'].isin(unique_ids)]

In [ ]:
# Get date of birth and admission time to calculate age
dob=patients[['SUBJECT_ID','DOB']]
d_admit=admissions[['SUBJECT_ID','ADMITTIME']]

# parsing string to datetime
from datetime import datetime
dob=pd.to_datetime(dob['DOB'],format='%Y-%m-%d %H:%M:%S')
d_admit=pd.to_datetime(d_admit['ADMITTIME'],format='%Y-%m-%d %H:%M:%S')

dob=pd.DataFrame(dob)
dob['subject_id']=patients[['SUBJECT_ID']] # adding back subject id

d_admit=pd.DataFrame(d_admit)
d_admit['subject_id']=admissions[['SUBJECT_ID']] # adding back subject id

# merging dob and d_admit dataframes based on subject id
age_mat=dob.merge(d_admit,how='inner',on='subject_id')

# calculating age from admit time and date of birth
age=age_mat['ADMITTIME'].sub(age_mat['DOB'])
age=age / np.timedelta64(1, 'Y') # convert to years

age=pd.DataFrame(age) 
age['subject_id'] =age_mat[['subject_id']] # adding back subject id

age=age.drop_duplicates(subset=['subject_id'])

check_neg=(age.loc[age[0]<0]) # check for negative age
print(check_neg)

pos_age=(age.loc[age[0]>0]) # only returning positive ages
pos_age=pos_age.rename(columns={'subject_id':'SUBJECT_ID',0:'Age'})
pos_age=pd.DataFrame([pos_age['SUBJECT_ID'], pos_age['Age']])
pos_age=pos_age.transpose()
pos_age


#plt.subplot(1,2,1)

# flattening all ages for plotting
#flat_all=age[0].ravel()
# histogram of age distribution with plot labels
#plt.hist(flat_all,bins=20)
#plt.title("Age distribution - neg included")
#plt.ylabel("Count")
#plt.xlabel("Age")


#plt.subplot(1,2,2)
# flattening positive age array for plotting
#flat_pos=pos_age[0].ravel()
#print(flat_pos)
# histogram of age distribution with plot labels
#plt.hist(flat_pos,bins=20)
#plt.title("Age distribution - pos only")
#plt.ylabel("Count")
#plt.xlabel("Age")

               0  subject_id
2807 -284.551699        6942


,SUBJECT_ID,Age
0,252.0,55.068026
2,261.0,76.396028
3,265.0,55.555698
4,267.0,24.748527
5,268.0,65.976904
...,...,...
3718,9984.0,80.000956
3720,9987.0,51.663298
3721,9991.0,70.478058
3722,9993.0,71.573570


In [ ]:
from sqlalchemy import create_engine

# Postgres username, password, database name
# @team - you'll need to change this according to what your Postgres info is under the Properties tab of your server in PgAdmin
POSTGRES_ADDRESS = '127.0.0.1'
POSTGRES_PORT= '5432'
POSTGRES_USERNAME= 'postgres'
POSTGRES_PASSWORD= 'password'
POSTGRES_DBNAME = 'mimic'

# putting in info
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

In [ ]:
# Height 920, 1394, 4187, 3486, 226707
height=pd.read_sql_query('''SELECT subject_id, value
FROM chartevents
WHERE itemid=920 or itemid=1394 or itemid=4187 or itemid= 3486 or itemid=226707;''',cnx)
height=height[height['subject_id'].isin(unique_ids)]
height=height.drop_duplicates()
height = height.replace(to_replace='None', value=np.nan).dropna()
height=height.rename(columns={'subject_id': 'SUBJECT_ID', 'value':'height'})
height

,SUBJECT_ID,height
286,36,71
301,124,68
307,188,68
309,188,70
311,209,65
...,...,...
174731,9982,60
174733,9984,65
174737,9987,66
174739,9983,68


In [ ]:
# Weight - patientweight from inputevents_mv
weight=pd.read_sql_query('''SELECT subject_id, patientweight FROM inputevents_mv;''',cnx)
weight=weight[weight['subject_id'].isin(unique_ids)]
weight=weight.drop_duplicates()
weight=weight.rename(columns={'subject_id': 'SUBJECT_ID'})
weight

NameError: name 'pd' is not defined

In [ ]:
# BMI
# add metric
weight_height=height.merge(weight,how='inner',on='SUBJECT_ID')
weight_height.patientweight=weight_height.patientweight/0.45359237
weight_height['height']=weight_height['height'].apply(lambda x: float(x))

weight_height['bmi']=703*((weight_height.patientweight)/weight_height.height**2)

weight_height=weight_height.rename(columns={'subject_id': 'SUBJECT_ID'})

weight_height

,SUBJECT_ID,height,patientweight,bmi
0,36,71.0,234.130922,32.651069
1,36,69.0,234.130922,34.571317
2,124,68.0,169.755942,25.808483
3,124,69.0,169.755942,25.065832
4,188,68.0,175.047036,26.612904
...,...,...,...,...
490,9356,67.0,158.732829,24.858360
491,9356,67.0,175.708423,27.516824
492,9356,67.0,178.574432,27.965655
493,9518,67.0,546.746410,85.623240


In [ ]:
# load in all the normalized chartevents
# load in all the normalized lab events
path = "/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/phys files/"
chart = pd.read_csv(r"/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/phys files/normalized_imputed_chart_mean_stats.csv")
lab = pd.read_csv(r"/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/phys files/normalized_imputed_lab_mean_stats.csv")

chart = chart.rename(columns={'subject_id':'SUBJECT_ID'}).drop(['file'], axis=1)
lab = lab.rename(columns={'subject_id':'SUBJECT_ID'}).drop(['file'], axis=1)

#Preparing dataframe for modeling

from functools import reduce
demog_merged=pd.DataFrame()
ind_frames = [ethnicities, marital_status, admit_loc,gender, insurance, pos_age]



demog_merged=reduce(lambda x,y: pd.merge(x,y,on='SUBJECT_ID',how='left'), ind_frames)
demog_merged=demog_merged.drop_duplicates(subset='SUBJECT_ID')

demog_merged

,SUBJECT_ID,ETHNICITY_ASIAN,ETHNICITY_ASIAN - ASIAN INDIAN,ETHNICITY_ASIAN - CHINESE,ETHNICITY_ASIAN - VIETNAMESE,ETHNICITY_BLACK/AFRICAN AMERICAN,ETHNICITY_BLACK/CAPE VERDEAN,ETHNICITY_HISPANIC OR LATINO,ETHNICITY_HISPANIC/LATINO - DOMINICAN,ETHNICITY_HISPANIC/LATINO - PUERTO RICAN,...,MARITAL_STATUS_WIDOWED,ADMISSION_LOCATION_CLINIC REFERRAL/PREMATURE,ADMISSION_LOCATION_EMERGENCY ROOM ADMIT,ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI,ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM,ADMISSION_LOCATION_TRANSFER FROM OTHER HEALT,ADMISSION_LOCATION_TRANSFER FROM SKILLED NUR,GENDER_F,GENDER_M,Age
0,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,72.005958
1,32,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,45.513141
2,36,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,69.699726
3,38,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,75.941380
4,41,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,56.634804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2592,9372,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,55.009879
2593,9374,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,78.125773
2594,9375,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,43.993730
2595,9377,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,81.928154


In [ ]:
# pulling y - death
expiry_flag=patients[['SUBJECT_ID','EXPIRE_FLAG']]

expiry_flag

,SUBJECT_ID,EXPIRE_FLAG
3,252,0
10,261,1
14,265,0
16,267,0
17,268,1
...,...,...
9842,9984,1
9844,9987,0
9848,9991,1
9850,9993,1


In [ ]:
test=expiry_flag.merge(demog_merged,on='SUBJECT_ID')
test=test.dropna()

In [ ]:
X=test.iloc[:, 2:]
y=test.iloc[:, 1]

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X,y)
LR.predict(X.iloc[460:,:])
round(LR.score(X,y), 4)

# add additional models: XGBOOST, SVM, RF etc
# plot AUROC, plot precision recall
# feature weights/importance


C:\Users\mnigc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6741

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

def get_preds(threshold, probabilities):
    return [1 if prob > threshold else 0 for prob in probabilities]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X, y)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X.iloc[460:,:])
    acc = accuracy_score(y, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X.iloc[460:,:])
    ll = log_loss(y, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)

    roc_values = []
    for thresh in np.linspace(0, 1, 100):
        preds = get_preds(thresh, probas)
        tn, fp, fn, tp = confusion_matrix(y,train_predictions  ).ravel()
        tpr = tp/(tp+fn)
        fpr = fp/(fp+tn)
        roc_values.append([tpr, fpr])
        tpr_values, fpr_values = zip(*roc_values)
    
    fig, ax = plt.subplots(figsize=(10,7))
    ax.plot(fpr_values, tpr_values)
    ax.plot(np.linspace(0, 1, 100),
         np.linspace(0, 1, 100),
         label='baseline',
         linestyle='--')
    plt.title('Receiver Operating Characteristic Curve', fontsize=18)
    plt.ylabel('TPR', fontsize=16)
    plt.xlabel('FPR', fontsize=16)
    plt.legend(fontsize=12);
    
    roc_auc_score(y, train_predictions)

print("="*30)